In [3]:
import pandas as pd

# Load CSV
df = pd.read_csv("data/lift_complaints.csv")

# Ensure the datetime column is parsed properly
df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'], errors='coerce')

# Clean the building address column
df['Incident Address'] = df['Incident Address'].astype(str).str.upper().str.strip()

# Drop exact duplicates of same address + same timestamp
df_deduped = df.drop_duplicates(subset=['Incident Address', 'Resolution Action Updated Date'])

# Save the cleaned dataset with full details
df_deduped.to_csv("deduped_complaints_with_details.csv", index=False)


/var/folders/s2/1r_lbpts7pvcd7p7h17w62tm0000gn/T/ipykernel_98699/1633889113.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Resolution Action Updated Date'] = pd.to_datetime(df['Resolution Action Updated Date'], errors='coerce')


In [4]:
print("Original rows:", len(df))
print("Filtered rows:", len(df_deduped))
print("Duplicates removed:", len(df) - len(df_deduped))


Original rows: 8563
Filtered rows: 5459
Duplicates removed: 3104


In [5]:
# Group by Incident Address + Timestamp
dupes = df.groupby(['Incident Address', 'Resolution Action Updated Date']).size().reset_index(name='count')

# Filter for entries with duplicate complaints
dupes = dupes[dupes['count'] > 1]

# See top duplicate groups
print(dupes.sort_values(by='count', ascending=False).head(10))


           Incident Address Resolution Action Updated Date  count
695      1325 JEROME AVENUE                     2025-03-06     21
3660       4215 PARK AVENUE                     2025-03-06     19
2359   247 WEST  145 STREET                     2025-04-11     18
3436   390 EAST  158 STREET                     2025-04-10     17
3256    3560 WEBSTER AVENUE                     2025-06-02     16
2408    2505 BEDFORD AVENUE                     2025-06-05     16
1609  1965 LAFAYETTE AVENUE                     2025-01-06     15
3469   3990 BRONX BOULEVARD                     2025-01-21     14
1008    150 LEFFERTS AVENUE                     2025-04-09     13
3253    3560 WEBSTER AVENUE                     2025-02-13     13


In [6]:
# Choose a suspicious address + timestamp from above
address = '764 EAST 152 STREET'
timestamp = '2025-06-14 00:00:00'

# Show all rows with this combo
dupe_rows = df[(df['Incident Address'] == address) &
               (df['Resolution Action Updated Date'] == pd.Timestamp(timestamp))]

print(dupe_rows)


Empty DataFrame
Columns: [Unique Key, Created Date, Closed Date, Agency, Agency Name, Complaint Type, Descriptor, Location Type, Incident Zip, Incident Address, Street Name, Cross Street 1, Cross Street 2, Intersection Street 1, Intersection Street 2, Address Type, City, Landmark, Facility Type, Status, Due Date, Resolution Description, Resolution Action Updated Date, Community Board, BBL, Borough, X Coordinate (State Plane), Y Coordinate (State Plane), Open Data Channel Type, Park Facility Name, Park Borough, Vehicle Type, Taxi Company Borough, Taxi Pick Up Location, Bridge Highway Name, Bridge Highway Direction, Road Ramp, Bridge Highway Segment, Latitude, Longitude, Location]
Index: []

[0 rows x 41 columns]


In [7]:
# Check for full-row duplicates
full_dupes = df[df.duplicated(subset=df.columns.tolist())]
print("Full-row duplicates:", len(full_dupes))


Full-row duplicates: 0


In [20]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process

# Load files
complaints_df = pd.read_csv("deduped_complaints_with_details.csv")
buildings_df = pd.read_csv("combined_output.csv")

# Step 1: Normalize addresses
def normalize_address(addr):
    addr = str(addr).upper()
    addr = re.sub(r'\bSTREET\b', 'ST', addr)
    addr = re.sub(r'\bAVENUE\b', 'AVE', addr)
    addr = re.sub(r'\bROAD\b', 'RD', addr)
    addr = re.sub(r'\bBOULEVARD\b', 'BLVD', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

buildings_df['full_address_clean'] = buildings_df['full_address'].apply(normalize_address)
complaints_df['address_clean'] = complaints_df['Incident Address'].apply(normalize_address)

# Step 2: Fuzzy match
address_lookup = buildings_df['full_address_clean'].tolist()

def fuzzy_match_address(addr, choices, threshold=90):
    match, score = process.extractOne(addr, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= threshold else None

complaints_df['matched_address'] = complaints_df['address_clean'].apply(
    lambda x: fuzzy_match_address(x, address_lookup)
)

# Step 3: Flag stabilized buildings
complaints_df['is_stabilized'] = complaints_df['matched_address'].notnull()

# Step 4: Save
complaints_df.to_csv("deduped_elevator_complaints_with_stabilized_flag_fuzzy.csv", index=False)
print(f"Done. Matched {complaints_df['is_stabilized'].sum()} out of {len(complaints_df)} complaints.")


Done. Matched 2941 out of 5459 complaints.


In [19]:
import pandas as pd
import re

# Load files
complaints_df = pd.read_csv("deduped_complaints_with_details.csv")
buildings_df = pd.read_csv("combined_output.csv")

# --------- 1. Clean and parse building addresses ----------
def extract_range_parts(addr):
    match = re.match(r'(\d+)(?: TO (\d+))?\s+(.*)', addr)
    if match:
        start = int(match.group(1))
        end = int(match.group(2)) if match.group(2) else start
        street = match.group(3).strip()
        return start, end, street
    return None, None, None

buildings_df['full_address'] = buildings_df['full_address'].str.upper().str.strip()
buildings_df[['start_num', 'end_num', 'street']] = buildings_df['full_address'].apply(
    lambda x: pd.Series(extract_range_parts(x))
)

# --------- 2. Clean and parse complaint addresses ----------
def extract_number_street(addr):
    match = re.match(r'(\d+)\s+(.*)', str(addr))
    if match:
        return int(match.group(1)), match.group(2).strip()
    return None, None

complaints_df['address_clean'] = complaints_df['Incident Address'].str.upper().str.strip()
complaints_df[['address_num', 'address_street']] = complaints_df['address_clean'].apply(
    lambda x: pd.Series(extract_number_street(x))
)

# --------- 3. Match within street groups ----------
# Initialize match result
complaints_df['is_stabilized'] = False

# Only match rows with valid numbers and streets
valid_complaints = complaints_df.dropna(subset=['address_num', 'address_street'])

# Group buildings by street
building_groups = buildings_df.groupby('street')

# Match by street
for street, group in valid_complaints.groupby('address_street'):
    if street not in building_groups.groups:
        continue
    bldgs = buildings_df.loc[building_groups.groups[street]]
    comp_idx = group.index
    comp_nums = group['address_num']

    # Build interval mask per building
    for idx in comp_idx:
        num = complaints_df.at[idx, 'address_num']
        matches = bldgs[(bldgs['start_num'] <= num) & (bldgs['end_num'] >= num)]
        if not matches.empty:
            complaints_df.at[idx, 'is_stabilized'] = True

# --------- 4. Save output ----------
complaints_df.to_csv("deduped_quickmatch.csv", index=False)
print(" Fast matching complete. Saved to 'elevator_complaints_with_stabilized_flag.csv'")


 Fast matching complete. Saved to 'elevator_complaints_with_stabilized_flag.csv'


In [ ]:
# NYCHA

In [18]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Step 1: Load elevator complaints (as GeoDataFrame)
complaints_df = pd.read_csv("deduped_complaints_with_details.csv")

# Drop rows without valid lat/lon
complaints_df = complaints_df.dropna(subset=["Latitude", "Longitude"])

# Create Point geometry from lat/lon
complaints_gdf = gpd.GeoDataFrame(
    complaints_df,
    geometry=[Point(xy) for xy in zip(complaints_df["Longitude"], complaints_df["Latitude"])],
    crs="EPSG:4326"  # WGS84
)

# Step 2: Load NYCHA Developments GeoData
nycha_df = pd.read_csv("data/NYCHA_Public_Housing_Developments.csv")

# Parse 'the_geom' column into shapely polygons (assuming WKT format)
from shapely import wkt
nycha_df['geometry'] = nycha_df['the_geom'].apply(wkt.loads)

# Convert to GeoDataFrame
nycha_gdf = gpd.GeoDataFrame(nycha_df, geometry='geometry', crs="EPSG:4326")

# Step 3: Spatial Join (complaint inside NYCHA polygon)
joined = gpd.sjoin(complaints_gdf, nycha_gdf, how="left", predicate="within")

# Step 4: Flag NYCHA matches
joined['is_nycha'] = joined['DEVELOPMEN'].notnull()

# Step 5: Save output
joined.to_csv("elevator_complaints_with_nycha_flag_spatial.csv", index=False)
print("Spatial join complete. Output saved to 'elevator_complaints_with_nycha_flag_spatial.csv'")


Spatial join complete. Output saved to 'elevator_complaints_with_nycha_flag_spatial.csv'
